In [7]:
%%configure -f
{
    "driverMemory": "2G",
    "executorMemory": "2G",
    "executorCores": 1,
    "numExecutors": 4
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1164,application_1765289937462_1157,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1136,application_1765289937462_1129,pyspark,idle,Link,Link,None,
1139,application_1765289937462_1132,pyspark,idle,Link,Link,None,
1153,application_1765289937462_1146,pyspark,idle,Link,Link,None,
1155,application_1765289937462_1148,pyspark,idle,Link,Link,None,
1156,application_1765289937462_1149,pyspark,idle,Link,Link,None,
1157,application_1765289937462_1150,pyspark,idle,Link,Link,None,
1158,application_1765289937462_1151,pyspark,idle,Link,Link,None,
1159,application_1765289937462_1152,pyspark,idle,Link,Link,None,
1162,application_1765289937462_1155,pyspark,idle,Link,Link,None,
1163,application_1765289937462_1156,pyspark,idle,Link,Link,None,


In [8]:
from pyspark.sql import SparkSession
import time
import csv
import io

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

print(f"Executors: {sc.getConf().get('spark.executor.instances')}")
print(f"Master: {sc.master}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executors: 4
Master: yarn

## Query 1
Να ταξινομημηθούν, σε φθίνουσα σειρά, οι ηλικιακές ομάδες των θυμάτων σε περιστατικά που περιλαμβάνουν οποιαδήποτε μορφή “βαριάς σωματικής βλάβης”. Θεωρείστε τις εξής ηλικιακές ομάδες:


• Παιδιά: < 18

• Νεαροί ενήλικοι: 18 – 24

• Ενήλικοι: 25 – 64

• Ηλικιωμένοι: >64


Ως εγκλήματα που περιλαμβάνουν οποιαδήποτε μορφή “βαριάς σωματικής βλάβης” θεωρούμε
όλα εκείνα τα περιστατικά που περιέχουν τον όρο “aggravated assault” στη σχετική περιγραφή.


In [9]:
crimes_old_path = "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv"
crimes_new_path = "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
CRM_CD_DESC_INDEX = 9
VICT_AGE_INDEX = 11

def map_to_age_groups(row):
    age = int(row[VICT_AGE_INDEX])
    if age is None:
        return None  
    if age < 18:
        age_group = "Children"
    elif 18 <= age <= 24:
        age_group = "Young Adults"
    elif 25 <= age <= 64:
        age_group = "Adults"
    else:
        age_group = "Elderly"
    return (age_group, 1)

def filter_assaults(row):
    return 'aggravated assault' in row[CRM_CD_DESC_INDEX].lower()

def parse_row(row_string):
    try:
        reader = csv.reader(io.StringIO(row_string))
        fields = next(reader)
        return fields
    except:
        return []


crimes_old_rdd = spark.sparkContext.textFile(crimes_old_path)
crimes_new_rdd = spark.sparkContext.textFile(crimes_new_path)
crimes_rdd = crimes_old_rdd.union(crimes_new_rdd).map(parse_row)

crimes_rdd.cache()
crimes_rdd.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3138130

In [11]:
runs = 10
times = []

for i in range(runs):

    assaults_rdd = crimes_rdd.filter(filter_assaults)

    counts_rdd = assaults_rdd \
        .map(map_to_age_groups) \
        .filter(lambda x: x is not None) \
        .reduceByKey(lambda a, b: a + b)

    sorted_rdd = counts_rdd.map(lambda x: (x[1], x[0])) \
                           .sortByKey(ascending=False)

    result_rdd = sorted_rdd.map(lambda x: (x[1], x[0]))

    start = time.time() 
    result = result_rdd.collect()
    end = time.time()

    time = end - start
    times.append(time)
    print(f"Run {i+1}: {time:.4f} sec")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'float' object has no attribute 'time'
Traceback (most recent call last):
AttributeError: 'float' object has no attribute 'time'



In [12]:
mean = sum(times) / runs
for age_group, count in result:
    print(f"- {age_group}: {count}")

print(f"Mean Execution Time: {mean} sec")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

- Adults: 121660
- Young Adults: 33758
- Children: 16014
- Elderly: 6011
Mean Execution Time: 0.04723775386810303 sec